<a href="https://colab.research.google.com/github/IsaiahHanna/Career-Paths/blob/main/ModelSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set Up Path and Directory

In [124]:
!git clone https://github.com/IsaiahHanna/Career-Paths.git
%cd Career-Paths

Cloning into 'Career-Paths'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 51 (delta 19), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 13.98 MiB | 11.16 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/Career-Paths/Career-Paths/Career-Paths/Career-Paths/Career-Paths


# Model Selection

#### Import Modules

In [125]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor,HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.preprocessing import MultiLabelBinarizer,StandardScaler
from sklearn.pipeline import Pipeline



#### Import Dataset

In [126]:
df = pd.read_csv("CareerPaths.csv")

In [127]:
stringCols = ['CREDENTIAL','GROUPBY','INSTITUTION','INSTITUTION_NAME','PROGRAM_AREA_NAME','PROGRAM_NAME']
for col in stringCols:
  if col == 'CREDENTIAL':
    encoded = pd.get_dummies(df[col], prefix=col)
    df = df.drop(col, axis=1)
    df = df.join(encoded)
  else:
    means = df.groupby(col)['SALARY_WAGE'].mean()
    df[col] = df[col].map(means)

for col in df.columns.tolist():
  if df[col].dtype == 'bool':
    df[col] = df[col].astype(int)

scaler = StandardScaler()

##### Split to testing and training sets

In [128]:
x = df.drop(columns = ["SALARY_WAGE",'PROGRAM_AREA'])
y = df['SALARY_WAGE']

In [129]:
xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.2,random_state = 42)

In [130]:
noNa = df.dropna()
x = noNa.drop(columns = ["SALARY_WAGE",'PROGRAM_AREA'])
y = noNa['SALARY_WAGE']
xTrainNoNa,xTestNoNa,yTrainNoNa,yTestNoNa = train_test_split(x,y,test_size=0.2,random_state = 42)
xTrainNoNa = scaler.fit_transform(xTrainNoNa)
xTestNoNa = scaler.transform(xTestNoNa)

### Test different models to see which is the best

#### Linear Regression

In [131]:
ln = LinearRegression()
ln.fit(xTrainNoNa,yTrainNoNa)
yPred = ln.predict(xTestNoNa)

In [132]:
mse = mean_squared_error(yTestNoNa,yPred)
mae = mean_absolute_error(yTestNoNa,yPred)
r2 = r2_score(yTestNoNa,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 10.208633461725793
Mean Absolute Error: 2.4407418117071225
R-squared: 0.7890143098210977


#### Lasso Regression

In [133]:
lasso = Lasso(alpha=0.1)
lasso.fit(xTrainNoNa,yTrainNoNa)
yPred = lasso.predict(xTestNoNa)

In [134]:
mse = mean_squared_error(yTestNoNa,yPred)
mae = mean_absolute_error(yTestNoNa,yPred)
r2 = r2_score(yTestNoNa,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 10.119444520265814
Mean Absolute Error: 2.394675503297902
R-squared: 0.7908576114187904


#### Ridge Regression

In [135]:
ridge = Ridge(alpha=0.1)
ridge.fit(xTrainNoNa,yTrainNoNa)
yPred = ridge.predict(xTestNoNa)

In [136]:
mse = mean_squared_error(yTestNoNa,yPred)
mae = mean_absolute_error(yTestNoNa,yPred)
r2 = r2_score(yTestNoNa,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 10.20913721291095
Mean Absolute Error: 2.44088669648748
R-squared: 0.7890038986047603


#### Decision Tree

In [137]:
regressor = DecisionTreeRegressor(random_state = 4)
regressor.fit(xTrain,yTrain)
yPred = regressor.predict(xTest)

In [138]:
mse = mean_squared_error(yTest,yPred)
mae = mean_absolute_error(yTest,yPred)
r2 = r2_score(yTest,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 14.463666115222205
Mean Absolute Error: 2.151268169660531
R-squared: 0.7906955116576067


##### Random Forest

In [139]:
regressor = RandomForestRegressor(n_estimators=10,oob_score=True,random_state=4)
regressor.fit(xTrain,yTrain)
yPred = regressor.predict(xTest)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [140]:
# Collect all relevant scores
oob_score = regressor.oob_score_
mse = mean_squared_error(yTest,yPred)
mae = mean_absolute_error(yTest,yPred)
r2 = r2_score(yTest,yPred)

print(f"Out-of-Bag Score: {oob_score}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")


Out-of-Bag Score: 0.696405879276258
Mean Squared Error: 8.162811892970302
Mean Absolute Error: 1.6959348497872133
R-squared: 0.8818755111544481


##### HistGradientBoostingRegressor

In [141]:
hist = HistGradientBoostingRegressor(random_state = 4)
hist.fit(xTrain,yTrain)
yPred = hist.predict(xTest)

In [142]:
mse = mean_squared_error(yTest,yPred)
mae = mean_absolute_error(yTest,yPred)
r2 = r2_score(yTest,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 7.3841707356699215
Mean Absolute Error: 1.7067176641767332
R-squared: 0.8931432691165574


#### Fine Tuning the Decision Tree Model


In [143]:
regressor = DecisionTreeRegressor(random_state = 42)

parameters = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(estimator=regressor, param_grid=parameters,
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1)


In [144]:
grid.fit(xTrain,yTrain)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error')

In [145]:
bestmodel = grid.best_estimator_
bestParams = grid.best_params_
yPred = bestmodel.predict(xTest)

In [146]:
print(f"Best Parameters (Grid Search): {bestParams}")

mse = mean_squared_error(yTest,yPred)
mae = mean_absolute_error(yTest,yPred)
r2 = r2_score(yTest,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Best Parameters (Grid Search): {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
Mean Squared Error: 10.407957548194371
Mean Absolute Error: 1.8900851541166508
R-squared: 0.8493858879235676


#### Fine Tuning the Ridge Regression Model


In [119]:
ridge = Ridge()
rfe = RFE(estimator=ridge, n_features_to_select=10)
xRFE = rfe.fit_transform(xTrainNoNa,yTrainNoNa)



In [120]:
parameters = {
    'alpha': [0.001,0.01, 0.1, 1, 10, 100]
}

grid = GridSearchCV(estimator = ridge,param_grid = parameters,cv = 5)

In [147]:
grid.fit(xRFE,yTrainNoNa)


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error')

In [150]:
bestModel = grid.best_estimator_
bestParams = grid.best_params_
yPred = bestModel.predict(rfe.transform(xTestNoNa))
print(f"Best Parameters (Grid Search): {bestParams}")
print(f"Selected Features: {rfe.support_}")

Best Parameters (Grid Search): {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
Selected Features: [False False False False  True  True False False  True  True  True False
 False False False False False False False False False False  True False
 False False False False False False  True  True  True  True]


In [151]:
mse = mean_squared_error(yTestNoNa,yPred)
mae = mean_absolute_error(yTestNoNa,yPred)
r2 = r2_score(yTestNoNa,yPred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 7.189333143890943
Mean Absolute Error: 1.757206820536756
R-squared: 0.8514153318387961
